In [85]:
import numpy as np
from pathlib import Path
import mne
from util import *
from scipy.signal import butter, sosfiltfilt, sosfreqz  # for filtering
import matplotlib.pyplot as plt   
import pandas as pd

In [86]:
np.random.seed(189)
rand_nums = np.random.permutation(6)

In [87]:
def butter_bandpass(lowcut, highcut, fs, order = 2):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        sos = butter(order, [low, high], analog = False, btype = 'band', output = 'sos')
        return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order = 2):
        sos = butter_bandpass(lowcut, highcut, fs, order = order)
        y = sosfiltfilt(sos, data)
        return y

In [88]:
# Path to GDF files
gdf_folder = Path("BCICIV_2a_gdf")

gdf_path = gdf_folder / "A01T.gdf"

# Users and target events
target_events = {'769': 'left', '770': 'right', '771': 'feet', '772': 'tongue'}

user_event_ids = {target_events[key]: event_id_map[key] for key in target_events if key in event_id_map}

# Load raw data and extract events
raw = mne.io.read_raw_gdf(gdf_path.resolve(), preload=True)
events, event_id_map = mne.events_from_annotations(raw)

Extracting EDF parameters from C:\Users\hunte\Documents\COGS189-BCI-Competition-Project\BCICIV_2a_gdf\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


c:\Users\hunte\anaconda3\envs\CSE156\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


In [96]:
passbands = [
    (1, 3), (2, 5), (4, 7), (6, 10), (7, 12),
    (10, 15), (12, 19), (15, 25), (19, 30),
    (25, 35), (30, 40)
]

target_events = {'769': 'left', '770': 'right', '771': 'feet', '772': 'tongue'}
user_event_ids = {target_events[key]: event_id_map[key] for key in target_events if key in event_id_map}

filtered_data = []
labels_2d = []

users = ["A01T.gdf", "A02T.gdf", "A03T.gdf", "A04T.gdf", "A05T.gdf", "A06T.gdf", "A07T.gdf", "A08T.gdf", "A09T.gdf"]

for subject in users:
    # Iterate through users
    gdf_path = gdf_folder / subject
    print(f"\nProcessing {subject}...")

    subject_data = []
    user_labels = []

    # Filter and epoch for each band
    for band in passbands:
        low, high = band
        print(f"  Filtering {low}-{high} Hz...")

        # Bandpass filter
        raw_filtered = raw.copy().filter(l_freq=low, h_freq=high, method='fir', fir_design='firwin')

        # Create epochs
        epochs = mne.Epochs(raw_filtered, events, event_id=user_event_ids, baseline=(None, 0), event_repeated='merge', preload=True, tmin=-1, tmax=4)
        labels = epochs.events[:, -1]  # Last column contains the event ID

        # Collect data and labels
        subject_data.append(epochs.get_data())  # Shape: (n_trials, n_channels, n_times)
        user_labels.append(labels)

    filtered_data.append(subject_data)
    labels_2d.append(user_labels)


Processing A01T.gdf...
  Filtering 1-3 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 3 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 3.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Filter length: 825 samples (3.300 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Not setting metadata
288 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1251 original time points ...
0 bad epochs dropped
  Filtering 2-5 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 5 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 5.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Not setting metadata
288 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1251 original time points ...
0 bad epochs dropped
  Filtering 4-7 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 7 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 7.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 8.00 Hz)
- Filter length: 413 samples (1.652 s)



...

Not setting metadata
288 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1251 original time points ...
0 bad epochs dropped
  Filtering 25-35 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 133 samples (0.532 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.9s


Not setting metadata
288 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1251 original time points ...
0 bad epochs dropped
  Filtering 30-40 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 30 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 30.00
- Lower transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 26.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 111 samples (0.444 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.8s


Not setting metadata
288 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1251 original time points ...
0 bad epochs dropped


In [97]:
filtered_data = np.array(filtered_data)
filtered_data.shape

(9, 11, 288, 25, 1251)

In [ ]:
# data_left = []
# data_right = []
# data_feet = []
# data_tongue = []

# for i in range(len(labels)):
#     if labels[i] == 7:
#         data_left.append(data[i])
#     elif labels[i] == 8:
#         data_right.append(data[i])
#     elif labels[i] == 9:
#         data_feet.append(data[i])
#     else:
#         data_tongue.append(data[i])

# data_left = np.array(data_left)
# data_right = np.array(data_right)
# data_feet = np.array(data_feet)
# data_tongue = np.array(data_tongue)

# data_left.shape, data_right.shape, data_feet.shape, data_tongue.shape

((72, 22, 1251), (72, 22, 1251), (72, 22, 1251), (72, 22, 1251))

In [44]:
labels

array([10,  9,  8,  7,  7,  8,  9, 10,  8,  9,  7,  7,  7, 10,  8,  8,  7,
        7,  9,  7,  8, 10, 10,  9,  7, 10, 10,  8, 10, 10,  8,  7,  8,  9,
        9,  9, 10,  9,  7, 10,  8,  9,  8,  9, 10,  8,  9,  7,  7,  7, 10,
        8,  7,  9,  7,  9,  8, 10,  7,  9,  9,  7,  9,  8, 10, 10, 10,  9,
        7, 10,  8, 10,  8,  7,  9,  8,  7,  9,  9,  7,  9, 10, 10,  8,  7,
        8, 10,  8, 10,  9,  8,  8,  8,  9, 10,  7,  8, 10,  7,  9,  9, 10,
        7,  7,  9,  8, 10, 10, 10,  8,  7,  9,  8, 10,  7, 10,  9,  8, 10,
       10,  7,  8,  8,  9, 10,  8,  7,  7, 10,  8,  7,  9,  8,  8,  9,  7,
       10,  9,  9,  9,  9,  7,  8,  7,  8,  7,  7,  9,  9,  8,  9, 10,  7,
       10,  7,  7,  8, 10,  9,  8, 10,  9, 10,  9, 10,  8,  8, 10,  7,  8,
        8,  8,  9, 10,  7, 10,  7,  9,  7, 10,  7,  9,  7,  8,  9,  9, 10,
        7,  8, 10,  8,  9,  9,  7, 10,  8, 10,  7,  7,  9,  9,  8, 10,  8,
        8,  7,  8, 10, 10,  8,  8,  8,  8, 10, 10,  9, 10,  7,  8,  9,  8,
        7, 10,  7, 10,  7

In [ ]:
#n_channel, n_time, n_sample = EEGR_train[0][0].shape
randIdx = np.random.permutation(n_sample)

In [4]:
k = 6
print(f'Our k is: {k}')

bin_size = int(n_sample / k)
print(f'Our bin size is: {bin_size}')

# In order to make our task easier, 
# we can reshape randIdx to have k rows and bin_size columns
# by using .reshape():
randIdx = randIdx.reshape(k, bin_size)
print(randIdx)
assert np.array_equal(randIdx,
                      np.array([[56, 70, 31, 50, 66, 34, 63,  4, 54,  9, 43, 53],
                                [52, 16, 14, 21, 10, 64, 41, 47, 28,  5, 20, 42],
                                [13, 33,  7, 59, 18, 71, 49, 22, 68, 26, 61, 32],
                                [ 6, 46,  3, 48, 44,  8, 11, 15, 17,  1, 24,  0],
                                [30, 65, 12,  2, 27, 37, 55, 23, 69, 67, 39, 57],
                                [36, 29, 45, 60, 38, 40, 51, 62, 19, 25, 58, 35]]))

Our k is: 6
Our bin size is: 12
[[56 70 31 50 66 34 63  4 54  9 43 53]
 [52 16 14 21 10 64 41 47 28  5 20 42]
 [13 33  7 59 18 71 49 22 68 26 61 32]
 [ 6 46  3 48 44  8 11 15 17  1 24  0]
 [30 65 12  2 27 37 55 23 69 67 39 57]
 [36 29 45 60 38 40 51 62 19 25 58 35]]


In [5]:
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int)
valIdx = np.zeros((k, bin_size), dtype=int)

In [6]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

array([[56, 70, 31, 50, 66, 34, 63,  4, 54,  9, 43, 53],
       [39, 59,  5,  7, 32, 36, 68,  9, 12, 60, 58, 10],
       [27, 35, 25, 63, 22, 49,  3, 20, 34, 33,  5, 28],
       [ 9, 36,  4, 18, 31, 54, 49, 30, 58, 42, 22, 34],
       [53, 34, 63, 10, 39, 61,  4, 51, 36, 69, 48, 23],
       [41,  6, 10, 23, 19, 29, 30, 59, 49, 39, 71, 51]])

In [7]:
# Depending on your solution, valIdx and trainIdx may not be np.arrays
# Let's guarantee they are
valIdx = np.array(valIdx)
trainIdx = np.array(trainIdx)

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

assert np.array_equal(valIdx[0,:], np.array([56, 70, 31, 50, 66, 34, 63,  4, 54,  9, 43, 53]))
assert np.array_equal(trainIdx[0,:], np.array([52, 16, 14, 21, 10, 64, 41, 47, 28,  5, 20, 42, 13, 33,  7, 59, 18,
                                               71, 49, 22, 68, 26, 61, 32,  6, 46,  3, 48, 44,  8, 11, 15, 17,  1,
                                               24,  0, 30, 65, 12,  2, 27, 37, 55, 23, 69, 67, 39, 57, 36, 29, 45,
                                               60, 38, 40, 51, 62, 19, 25, 58, 35]))

In [9]:
# Main Analysis (Do Not Modify)

# Declare useful variables
n_subjects, n_filters = EEGL_train.shape
train_size = bin_size * (k-1)
val_size = bin_size
accuracy = np.zeros((n_subjects, k))
csp_per_class = 3
trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
# Subject > Fold > Band
print('===============')
for subject in range(n_subjects):
    print('Processing Subject {}/{}'.format(subject+1, n_subjects))
    for fold in range(k):
        print('Processing Fold {}/{}'.format(fold+1, k))
        band_score_train = np.zeros((train_size*2, n_filters))
        band_score_val = np.zeros((val_size*2, n_filters))
        for band in range(n_filters):
            # train,val dataset -> train,val for each subject&band
            L_train = EEGL_train[subject, band][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            R_train = EEGR_train[subject, band][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            L_val = EEGL_train[subject, band][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            R_val = EEGR_train[subject, band][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            train_data = np.stack((L_train,R_train), axis=0) # <- CSP input data format (classes, trials, chans, samples) 
            val_data = np.stack((L_val,R_val), axis=0) # <- CSP input data format (classes, trials, chans, samples)

            # Run Mahta's CSP code
            # Source: https://github.com/mahtamsv/CCACSP
            csp_filts, clf = train(train_data[0], train_data[1], csp_per_class)
            train_prob = test(np.vstack(train_data), csp_filts, clf)
            val_prob = test(np.vstack(val_data), csp_filts, clf)
            
            # test() gives the prob of the 0th class, that's why 
            # we have y_val seemingly switched below
            y_val = np.append(np.ones(12), np.zeros(12))

            band_score_train[:,band] = train_prob
            band_score_val[:,band] = val_prob
        
        
        accuracy[subject,fold] = sum(np.rint(band_score_val.mean(1))==y_val)/len(y_val)
    print('===============')
print(np.mean(accuracy, axis=1))

# Main Analysis Over

Processing Subject 1/9
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 2/9
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 3/9
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 4/9
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 5/9
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 6/9
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 7/9
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6